In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset
from transformers import AutoTokenizer

import random
import math
import time
from tqdm import tqdm

In [4]:
class EncoderRNN(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):

        embedded = self.dropout(self.embedding(src))

        outputs, (hidden, cell) = self.rnn(embedded)
       
        return outputs, hidden, cell

In [5]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hid_dim) + dec_hid_dim, dec_hid_dim) 
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
       
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        hidden = hidden.squeeze(0).unsqueeze(0).repeat(src_len, 1, 1)

        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))

        attention = self.v(energy).squeeze(2)

        return torch.softmax(attention, dim=0)

In [6]:
class DecoderRNN(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim + hid_dim, hid_dim, dropout=dropout) 
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim) 
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell, encoder_outputs):
       
        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        attention_weights = self.attention(hidden, encoder_outputs)

        attention_weights = attention_weights.permute(1, 0)

        attention_weights = attention_weights.unsqueeze(1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_context = torch.bmm(attention_weights, encoder_outputs).squeeze(1)

        weighted_context = weighted_context.unsqueeze(0)

        rnn_input = torch.cat((embedded, weighted_context), dim=2)

        output, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_context = weighted_context.squeeze(0) # Squeeze again after unsqueeze(0) for concat

        prediction = self.fc_out(torch.cat((output, weighted_context, embedded), dim=1))
        
        return prediction, hidden, cell, attention_weights.squeeze(1)

In [7]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.rnn.hidden_size, \
            "Hidden dimensions of encoder and decoder must be equal!"

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
      

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden, cell = self.encoder(src)

        input = trg[0, :]

        for t in range(1, trg_len):
            # Get output prediction from decoder
            output, hidden, cell, _ = self.decoder(input, hidden, cell, encoder_outputs)

            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1)

            input = trg[t, :] if teacher_force else top1

        return outputs

In [8]:
def load_cnn_dailymail_dataset(num_train_samples=None, num_val_samples=None):
    print("Loading CNN/DailyMail dataset...")
    dataset = load_dataset("cnn_dailymail", '2.0.0') 

    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") 

    tokenizer.add_special_tokens({'bos_token': '<sos>', 'eos_token': '<eos>'})

    def tokenize_function(examples):
        # Tokenize articles
        model_inputs = tokenizer(examples['article'], max_length=512, truncation=True, padding="max_length")
        labels = tokenizer(examples['highlights'], max_length=128, truncation=True, padding="max_length")

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    print("Tokenizing dataset...")
    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    if num_train_samples is not None:
        tokenized_datasets['train'] = tokenized_datasets['train'].select(range(num_train_samples))
        print(f"Using {len(tokenized_datasets['train'])} training samples.")
    if num_val_samples is not None:
        tokenized_datasets['validation'] = tokenized_datasets['validation'].select(range(num_val_samples))
        print(f"Using {len(tokenized_datasets['validation'])} validation samples.")

    tokenized_datasets = tokenized_datasets.remove_columns(["article", "highlights", "id"])
    tokenized_datasets.set_format("torch")

    return tokenized_datasets, tokenizer

In [9]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for batch in tqdm(iterator, desc="Training"):
        src = batch['input_ids'].transpose(0, 1).to(model.device) 
        trg = batch['labels'].transpose(0, 1).to(model.device)  

        optimizer.zero_grad()

        output = model(src, trg)

        output_dim = output.shape[-1]

        output = output[1:].reshape(-1, output_dim)
        trg = trg[1:].reshape(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [10]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for batch in tqdm(iterator, desc="Evaluating"):
            src = batch['input_ids'].transpose(0, 1).to(model.device) 
            trg = batch['labels'].transpose(0, 1).to(model.device)  

            output = model(src, trg, 0) 

            output_dim = output.shape[-1]

            output = output[1:].reshape(-1, output_dim)
            trg = trg[1:].reshape(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

NUM_TRAIN = 300
NUM_VAL = 50
BATCH_SIZE = 4
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_EPOCHS = 10
CLIP = 1.0

tokenized_datasets, tokenizer = load_cnn_dailymail_dataset(num_train_samples=NUM_TRAIN, num_val_samples=NUM_VAL)

train_iterator = DataLoader(tokenized_datasets['train'], batch_size=BATCH_SIZE, shuffle=True)
valid_iterator = DataLoader(tokenized_datasets['validation'], batch_size=BATCH_SIZE)

INPUT_DIM = len(tokenizer)
OUTPUT_DIM = len(tokenizer)
TRG_PAD_IDX = tokenizer.pad_token_id

attention = Attention(HID_DIM, HID_DIM) 
encoder = EncoderRNN(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
decoder = DecoderRNN(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT, attention)

model = Seq2Seq(encoder, decoder, device).to(device)

Using device: cuda
Loading CNN/DailyMail dataset...
Tokenizing dataset...
Using 300 training samples.
Using 50 validation samples.


In [27]:
def init_weights(m):
        for name, param in m.named_parameters():
            if 'weight' in name:
                nn.init.normal_(param.data, mean=0, std=0.01)
            else:
                nn.init.constant_(param.data, 0)


In [28]:
model.apply(init_weights)

Seq2Seq(
  (encoder): EncoderRNN(
    (embedding): Embedding(30524, 256)
    (rnn): LSTM(256, 512, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): DecoderRNN(
    (attention): Attention(
      (attn): Linear(in_features=1024, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(30524, 256)
    (rnn): LSTM(768, 512, dropout=0.5)
    (fc_out): Linear(in_features=1280, out_features=30524, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [29]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX) # Ignore padding index in loss calculation

best_valid_loss = float('inf')

In [30]:
for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins:.0f}m {epoch_secs:.0f}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):.3f}')

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best_seq2seq_attn_model.pt')

print("Training finished.")

Evaluating: 100%|██████████| 13/13 [00:03<00:00,  3.79it/s]


Epoch: 01 | Time: 1m 25s
	Train Loss: 8.316 | Train PPL: 4088.700
	 Val. Loss: 7.734 |  Val. PPL: 2285.060


Evaluating: 100%|██████████| 13/13 [00:03<00:00,  3.71it/s]


Epoch: 02 | Time: 1m 25s
	Train Loss: 6.934 | Train PPL: 1026.566
	 Val. Loss: 7.873 |  Val. PPL: 2624.589


Evaluating: 100%|██████████| 13/13 [00:03<00:00,  3.68it/s]


...

Epoch: 08 | Time: 1m 25s
	Train Loss: 5.990 | Train PPL: 399.382
	 Val. Loss: 8.440 |  Val. PPL: 4629.043


Evaluating: 100%|██████████| 13/13 [00:03<00:00,  3.67it/s]


Epoch: 09 | Time: 1m 25s
	Train Loss: 5.768 | Train PPL: 319.902
	 Val. Loss: 8.540 |  Val. PPL: 5115.242


Evaluating: 100%|██████████| 13/13 [00:03<00:00,  3.66it/s]

Epoch: 10 | Time: 1m 25s
	Train Loss: 5.588 | Train PPL: 267.166
	 Val. Loss: 8.712 |  Val. PPL: 6077.669
Training finished.
